# Célula 1 — Instalação de pacotes

O que faz:
- Executa `pip` para garantir que `pandas`, `matplotlib`, `seaborn` e `scipy` estejam instalados no ambiente do notebook.

Observação: útil quando o notebook é aberto em um ambiente limpo. Em ambientes já configurados essa célula pode ser removida ou comentada.

In [2]:
# EXERCÍCIOS PŔE PROVA
# Autor: Fernanda Martins da Silva BV3032345
!pip install pandas matplotlib seaborn scipy --quiet


# Célula 2 — Importação e carregamento do CSV

O que faz:
- Importa `os`, `pandas` e `numpy`.
- Lista os arquivos do diretório de trabalho para confirmar o path do CSV.
- Lê o arquivo `video-game-sales-with-ratings/vgsales.csv` para o DataFrame `df` e exibe informações iniciais (shape, head, colunas e `df.info()`).

Entradas/Saídas: entrada = arquivo CSV; saída = DataFrame `df` e resumo impresso dos tipos e nulos.

Observação: verifica se o caminho do CSV está correto antes de prosseguir com a análise.

In [ ]:
import os
import pandas as pd
import numpy as np

# mostrar arquivos na pasta atual para confirmar o CSV
print(os.listdir('.'))

# carregar (ajuste o nome do arquivo se necessário)
df = pd.read_csv('video-game-sales-with-ratings/vgsales.csv')

# visão geral
print(df.shape)
display(df.head())
display(df.columns.tolist())
df.info()


# Célula 3 — Conversão de tipos e tratamento de valores especiais

O que faz:
- Converte `Year`, `Critic_Score`, `User_Score` (tratando 'tbd') e `Global_Sales` para tipos numéricos usando `pd.to_numeric(..., errors='coerce')`.
- Substitui `'tbd'` em `User_Score` por `NaN` antes da conversão.
- Mostra a contagem de valores faltantes nas colunas importantes para orientar limpeza adicional.

Porque é importante: tipos corretos são necessários para estatísticas e plots numéricos. `errors='coerce'` transforma entradas inválidas em `NaN` — útil para detectar problemas.

In [ ]:
# converter Year para numérico
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# tratar User_Score (alguns datasets usam 'tbd' para "to be determined")
df['User_Score'] = df['User_Score'].replace('tbd', np.nan)
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

# Critic_Score -> numérico
df['Critic_Score'] = pd.to_numeric(df['Critic_Score'], errors='coerce')

# Global_Sales para numérico (em milhões)
df['Global_Sales'] = pd.to_numeric(df['Global_Sales'], errors='coerce')

# contar valores faltantes nas colunas importantes
cols = ['Name','Platform','Year','Genre','Publisher','Global_Sales','Critic_Score','User_Score']
display(df[cols].isna().sum())


# Célula 4 — Probabilidades e probabilidades condicionais

O que faz:
- Calcula várias probabilidades simples e condicionais com base nas colunas categóricas e numéricas do DataFrame:
  - P(gênero = 'Action')
  - P(publisher = 'Nintendo')
  - P(Genre='Sports' | Platform='Wii') (condicional)
  - P(Genre='Role-Playing' e Publisher='Square Enix') (conjunta)
  - P(Critic_Score >= 90) — jogos 'aclamados'

Observações: todas as proporções evitam `NaN` utilizando `.dropna()` e tratam casos com contagem zero.

In [ ]:
N_total = len(df)

# 1) Probabilidade de ser do gênero "Action"
p_action = df['Genre'].dropna().eq('Action').sum() / df['Genre'].dropna().shape[0]

# 2) Probabilidade de publisher "Nintendo"
p_nintendo = df['Publisher'].dropna().eq('Nintendo').sum() / df['Publisher'].dropna().shape[0]

# 3) Prob condicional: P(Genre='Sports' | Platform='Wii')
df_wii = df[df['Platform'] == 'Wii']
if len(df_wii) > 0:
    p_sports_given_wii = df_wii['Genre'].dropna().eq('Sports').sum() / df_wii['Genre'].dropna().shape[0]
else:
    p_sports_given_wii = np.nan

# 4) Probabilidade de Role-Playing AND Publisher Square Enix
joint = df[(df['Genre'] == 'Role-Playing') & (df['Publisher'] == 'Square Enix')]
p_joint = joint.shape[0] / N_total

# 5) Probabilidade de ser "aclamado" (Critic_Score >= 90)
crit_valid = df['Critic_Score'].dropna()
p_aclamado = (crit_valid >= 90).sum() / crit_valid.shape[0]

print(f"P(genêro=Action) = {p_action:.4f}")
print(f"P(publisher=Nintendo) = {p_nintendo:.4f}")
print(f"P(Genre=Sports | Platform=Wii) = {p_sports_given_wii:.4f}")
print(f"P(Role-Playing e Publisher=Square Enix) = {p_joint:.6f}")
print(f"P(Critic_Score >= 90) = {p_aclamado:.4f}")


# Célula 5 — Estatística descritiva (média, mediana, moda, variância)

O que faz:
- Calcula média, mediana e moda de `Global_Sales` e dá uma interpretação simples comparando média vs mediana (dica sobre skew).
- Calcula variância e desvio padrão (populacional e amostral) para `Critic_Score` e `User_Score`.

Observação: a moda pode retornar vários valores; a distinção entre ddof=0 e ddof=1 é didática (populacional vs amostral).

In [ ]:
# Global_Sales: média, mediana, moda
gs = df['Global_Sales'].dropna()
mean_gs = gs.mean()
median_gs = gs.median()
mode_gs = gs.mode()   # pode ter mais de um

print("Global_Sales — mean:", mean_gs, "median:", median_gs, "mode(s):", list(mode_gs))

# Diferença média vs mediana -> interpretação
if mean_gs > median_gs:
    skew_hint = "direita (cauda à direita) — alguns jogos com vendas muito altas puxam a média para cima"
elif mean_gs < median_gs:
    skew_hint = "esquerda"
else:
    skew_hint = "simétrica (média ~ mediana)"

print("Interpretação: distribuição tende a", skew_hint)

# Variância e desvio padrão — Critic_Score e User_Score
from math import sqrt
for col in ['Critic_Score','User_Score']:
    s = df[col].dropna()
    var_pop = s.var(ddof=0)   # variância populacional
    std_pop = s.std(ddof=0)
    var_samp = s.var(ddof=1)  # amostral (padrão pandas .var() usa ddof=1)
    std_samp = s.std(ddof=1)
    print(f\
    print(f\
)
    print(f\
)
    print(f\
)


# Célula 6 — Detecção de outliers por IQR (Global_Sales)

O que faz:
- Calcula Q1, Q3 e IQR; define limites inferior e superior (Q1 - 1.5*IQR, Q3 + 1.5*IQR).
- Identifica linhas cujo `Global_Sales` está fora desses limites e mostra as top ocorrências por vendas globais.

Observação: identifica jogos com vendas excepcionalmente altas/baixas; útil para decidir transformações ou exclusões.

In [ ]:
Q1 = gs.quantile(0.25)
Q3 = gs.quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = df[(df['Global_Sales'] < lower) | (df['Global_Sales'] > upper)]
print("IQR:", IQR, "lower:", lower, "upper:", upper)
print("Número de outliers (Global_Sales):", outliers.shape[0])
display(outliers[['Name','Platform','Genre','Global_Sales']].sort_values('Global_Sales', ascending=False).head(10))


# Célula 7 — Histogramas e distribuição (Global_Sales, log e Critic_Score)

O que faz:
- Plota histograma linear de `Global_Sales` e histograma de `log1p(Global_Sales)` (para visualizar melhor a forma quando há muitos outliers).
- Plota histograma de `Critic_Score` com KDE.

Observação: `log1p` é usado para lidar com zeros; ajuda a enxergar a estrutura central da distribuição.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# Histograma Global_Sales (linear)
plt.figure(figsize=(8,4))
sns.histplot(gs, bins=50, kde=False)
plt.title("Global_Sales (linear)")
plt.xlabel("Global_Sales (milhões)")
plt.show()

# Global_Sales em log para ver a "forma"
plt.figure(figsize=(8,4))
sns.histplot(np.log1p(gs), bins=50, kde=True)
plt.title("log1p(Global_Sales)")
plt.xlabel("log1p(Global_Sales)")
plt.show()

# Critic_Score histograma (0-100)
plt.figure(figsize=(8,4))
sns.histplot(df['Critic_Score'].dropna(), bins=30, kde=True)
plt.title("Critic_Score")
plt.show()


# Célula 8 — Rankings e contagens (Top 10, plataformas e gêneros)

O que faz:
- Mostra os Top 10 jogos por `Global_Sales`.
- Conta o número de jogos por `Platform` e exibe as principais (top 15).
- Conta gêneros mais comuns e exibe os principais (top 15).

Observação: ajuda a identificar títulos/plataformas/gêneros dominantes no dataset.

In [ ]:
# 1) Top 10 jogos mais vendidos
top10 = df.sort_values('Global_Sales', ascending=False).head(10)
display(top10[['Name','Platform','Genre','Global_Sales']])

# 2) Plataformas com mais jogos lançados
platform_counts = df['Platform'].value_counts()
display(platform_counts.head(15))

# 3) Gêneros mais comuns
genre_counts = df['Genre'].value_counts()
display(genre_counts.head(15))


# Célula 9 — Correlações (Pearson e Spearman)

O que faz:
- Define a função `correlation_report` que calcula Pearson (linear) e Spearman (ordem) entre dois atributos, mostrando coeficiente e p-value.
- Executa o relatório para `Critic_Score` vs `Global_Sales` e `User_Score` vs `Global_Sales`.

Observação: Pearson avalia relação linear; Spearman avalia associação monotônica. A função verifica se há dados suficientes antes de calcular.

In [ ]:
from scipy.stats import pearsonr, spearmanr

def correlation_report(a,b,labelA,labelB):
    df_pair = df[[a,b]].dropna()
    x = df_pair[a]
    y = df_pair[b]
    if len(df_pair) < 3:
        print(f"{labelA} vs {labelB}: poucos dados")
        return
    pr, pval_pr = pearsonr(x,y)
    sr, pval_sr = spearmanr(x,y)
    print(f"{labelA} vs {labelB}: N={len(df_pair)}")
    print(f"  Pearson r = {pr:.4f}, p-value = {pval_pr:.4g}")
    print(f"  Spearman rho = {sr:.4f}, p-value = {pval_sr:.4g}")
    print()

# Critic_Score vs Global_Sales
correlation_report('Critic_Score','Global_Sales','Critic_Score','Global_Sales')

# User_Score vs Global_Sales
correlation_report('User_Score','Global_Sales','User_Score','Global_Sales')


# Célula 10 — Vendas totais por gênero (barras)

O que faz:
- Agrupa por `Genre` somando `Global_Sales` e plota um gráfico de barras com os totais por gênero.

Observação: mostra quais gêneros contribuíram mais para o total de vendas no dataset.

In [ ]:
sales_by_genre = df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False)
plt.figure(figsize=(10,5))
sns.barplot(x=sales_by_genre.index, y=sales_by_genre.values)
plt.xticks(rotation=45, ha='right')
plt.ylabel('Total Global Sales (milhões)')
plt.title('Vendas globais por gênero')
plt.show()


# Célula 11 — Número de jogos por ano (série temporal)

O que faz:
- Conta o número de jogos por `Year` (após conversão para numérico) e plota uma série temporal com o número de lançamentos por ano.

Observação: `Year` com `NaN` será descartado; útil para ver tendências históricas de publicações.

In [ ]:
games_per_year = df.groupby('Year').size().dropna().astype(int)
plt.figure(figsize=(10,4))
games_per_year.plot()
plt.xlabel('Ano')
plt.ylabel('Número de jogos lançados')
plt.title('Número de jogos por ano')
plt.show()


# Célula 12 — Regressão simples (Critic_Score vs Global_Sales)

O que faz:
- Plota um scatter entre `Critic_Score` e `Global_Sales` e ajusta uma linha de tendência (regressão) com `sns.regplot`.

Observação: visualmente avalia se notas de crítica estão associadas a maiores vendas; atenção a outliers que podem influenciar a reta.

In [ ]:
plt.figure(figsize=(7,5))
sns.regplot(x='Critic_Score', y='Global_Sales', data=df, scatter_kws={'alpha':0.4, 's':20}, line_kws={'color':'red'})
plt.title('Critic_Score vs Global_Sales (com linha de tendência)')
plt.show()


# Célula 13 — Boxplot de Global_Sales por plataforma (top 5)

O que faz:
- Seleciona as 5 plataformas com mais jogos, plota boxplots de `Global_Sales` por plataforma e aplica escala `symlog` no eixo y para reduzir efeito de outliers.

Observação: `symlog` preserva zeros e sinais; alternativa comum é transformar a coluna via `log1p` antes do plot.

In [ ]:
# escolher top plataformas por contagem
top_platforms = df['Platform'].value_counts().head(5).index.tolist()
df_topplat = df[df['Platform'].isin(top_platforms)]

plt.figure(figsize=(8,5))
sns.boxplot(x='Platform', y='Global_Sales', data=df_topplat)
plt.yscale('symlog')  # escala log simétrica para lidar com outliers (opcional)
plt.title('Distribuição de Global_Sales por plataforma (top 5)')
plt.show()
